In [1]:
import pickle

import numpy as np
import polars as pl

from bm25 import BM25Model

In [2]:
features = pl.read_parquet('./features.parquet', columns = ['video_id', 'v_pub_datetime']).sort('v_pub_datetime')
features.tail()

video_id,v_pub_datetime
str,str
"""video_2172262""","""2023-11-22 15:…"
"""video_22618974…","""2023-11-22 15:…"
"""video_11792065…","""2023-11-22 15:…"
"""video_11792064…","""2023-11-22 15:…"
"""video_11792080…","""2023-11-22 15:…"


In [3]:
features = features.unique()

In [4]:
videos = pl.read_parquet('./videos.parquet', columns = ['video_id', 'video_title', 'v_pub_datetime']).sort('v_pub_datetime')
videos.tail(1_000_000)

video_id,video_title,v_pub_datetime
str,str,"datetime[ns, +00:00]"
"""video_27609121…","""LIVE 3 : Gayat…",2023-11-12 22:53:48 +00:00
"""video_16206525…","""6 - Шримати Ра…",2023-11-12 22:53:49 +00:00
"""video_15284998…","""7 CHAKRAS HEAL…",2023-11-12 22:53:49 +00:00
"""video_13865933…","""хадсон — Море…",2023-11-12 22:53:49 +00:00
"""video_18611538…","""KALIYUGA || MO…",2023-11-12 22:53:49 +00:00
"""video_5992598""","""Dmitri Hvorost…",2023-11-12 22:53:49 +00:00
"""video_21456410…","""नवरात्रि स्पॆश…",2023-11-12 22:53:49 +00:00
"""video_32520621…","""38 лет в браке…",2023-11-12 22:53:49 +00:00
"""video_10561672…","""Большое спасиб…",2023-11-12 22:53:49 +00:00


за последние два дня целых миллион видео???

In [5]:
videos = videos.join(features, on='video_id', how='left')
videos.tail()

video_id,video_title,v_pub_datetime,v_pub_datetime_right
str,str,"datetime[ns, +00:00]",str
"""video_14412320…","""Угадай мелодию…",2023-11-14 11:58:56 +00:00,null
"""video_15300130…","""Гога, Киря и Е…",2023-11-14 11:59:13 +00:00,null
"""video_15600786…","""Специальный ре…",2023-11-14 11:59:26 +00:00,null
"""video_15299879…","""Угадай мелодию…",2023-11-14 11:59:41 +00:00,null
"""video_11792077…","""Давай поженимс…",2023-11-14 11:59:59 +00:00,null


In [6]:
videos = videos.filter((~pl.col('v_pub_datetime').is_null()) & (~pl.col('v_pub_datetime_right').is_null()))
videos.tail(1_000_000)

video_id,video_title,v_pub_datetime,v_pub_datetime_right
str,str,"datetime[ns, +00:00]",str
"""video_27711768…","""Авто из Кореи …",2023-09-27 10:30:42 +00:00,"""2023-09-27 10:…"
"""video_16046805…","""Русский язык_5…",2023-09-27 10:30:43 +00:00,"""2023-09-27 10:…"
"""video_32024614…","""Apera Instrume…",2023-09-27 10:30:43 +00:00,"""2023-09-27 10:…"
"""video_28578316…","""СМОЖЕШЬ СДЕЛАТ…",2023-09-27 10:30:44 +00:00,"""2023-09-27 10:…"
"""video_370606""","""Жди Меня 05.01…",2023-09-27 10:30:44 +00:00,"""2023-09-27 10:…"
"""video_15732705…","""Сергей Банарь …",2023-09-27 10:30:44 +00:00,"""2023-09-27 10:…"
"""video_24676052…","""Вопрос №2 § 27…",2023-09-27 10:30:45 +00:00,"""2023-09-27 10:…"
"""video_5606659""","""Как Сделать Ро…",2023-09-27 10:30:46 +00:00,"""2023-09-27 10:…"
"""video_7076222""","""ПАРА ТУФЕЛЬ. Д…",2023-09-27 10:30:47 +00:00,"""2023-09-27 10:…"


теперь выглядит более реалистично

In [7]:
video_titles = videos.tail(500_000)["video_title"].to_list()

In [8]:
bm25_model = BM25Model(100)

In [9]:
bm25_model.fit(video_titles)

  0%|          | 0/500000 [00:00<?, ?it/s]

In [10]:
%%time
ids, _ = bm25_model.predict('цифровой прорыв', 10)

CPU times: user 472 ms, sys: 16.4 ms, total: 488 ms
Wall time: 486 ms


In [11]:
np.array(video_titles)[ids]

array(['Стартовал новый сезона проекта "Цифровой прорыв. Сезон: искусственный интеллект"',
       'Genshin Impact: Прорыв вдохновения',
       'Halo 5 Guardians | Прохождение #7 | Прорыв',
       'Грехо-Обзор "Вторжение" (Прорыв канализации в Москве)',
       'ПРОРЫВ В БЛАГОУСТРОЙСТВЕ СКВЕР ТЕКСТИЛЬНАЯ ЛОБНЯ',
       'Цифровой манометр', 'Цифровой полис', 'Цифровое расширение!',
       'Цифровой паспорт', 'Цифровой акробат))'], dtype='<U100')

In [12]:
with open('bm25_model.pickle', 'wb') as f:
    pickle.dump(bm25_model, f)